# Anàlisi del text

Intro

In [1]:
import pandas as pd 
filename = 'tocho2.csv'
df = pd.read_csv(filename, index_col = 0)
df['text_ tokens'] = df['text_ tokens'].str.split('\t')
print(df['text_ tokens'].iloc[0])

['101', '16493', '12478', '117', '10105', '42370', '76299', '100', '187', '15480', '10108', '23837', '117', '10393', '169', '32342', '18077', '10135', '10226', '27925', '10111', '10393', '17087', '10950', '10114', '41904', '51937', '16981', '10142', '15217', '119', '14820', '10689', '17860', '169', '28502', '11360', '10689', '13461', '10105', '13451', '35688', '26981', '10142', '12557', '136', '14120', '131', '120', '120', '188', '119', '11170', '120', '175', '11166', '15705', '10129', '19282', '10113', '11396', '10362', '102']


Volem identificar l'idioma dels twits, codificats en [BERT Base Multilingual Cased](https://huggingface.co/bert-base-multilingual-cased). Com els valor de la columna language estan codificats, volem inspeccionar el contingut dels textos per trobar els twits escrits en llengües que poguem interpretar. 

Per això decodifiquem el text d'un twit per valor de `language` i llegim el seu contingut. Farem servir la lliberia *transformers*:

In [2]:
#Import of the BertTokenizer module to detokenize the text tokens
from transformers import BertTokenizer

#Setting the BERT Base Multilingual Cased tokenizer model
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

#Print the first tweet in the most frequent languages
for index in df.language.value_counts().index[:21]:
    dl = df[df.language == index].copy()
    dl = dl.reset_index(drop=True)
    print("language = {} \n".format(index))        
    print(tokenizer.decode(dl['text_ tokens'].iloc[0]) + '\n')

language = D3164C7FBCF2565DDF915B1B3AEFB1DC 

[CLS] Jacob Young, the Syndicate [UNK] s Director of PR, has a serious problem on his hands and has reached out to Hammond Robotics for help. Can they find a solution before they become the next ones marked for death? https : / / t. co / g5evrQa8va [SEP]

language = 22C448FF81263D4BAF2A176145EE9EAD 

[CLS] ただいま うさ 顔 洗 い 中. おしっこ 飛 ばしの 後 にはいつもする 仕 草. 壁 跡 は 生 々しいけれど 、 なんとも 愛 らしい [UNK] [UNK] ！.. # うさぎ # うさぎ 好 きさんと 繋 がりたい # 顔 洗 い # キリトリセカイ # 癒 し 動 画 # 動 物 好 きな 人 と 繋 がりたい # うさぎあるある # 動 物 あるある # ファインダー 越 しの 私 の 世 界 # 部 屋 んぽ # かわいい https : / / t. co / 2gKb0EeWqa [SEP]

language = 06D61DCBBE938971E1EA0C38BD9B5446 

[CLS] RT @ GermantequilIa : otra vez Las Palmas hacia ruta 68 pista izquierda bloqueada por choque de vehículo @ sitiodelsuceso @ AlertaVR https : / / t. [UNK] [SEP]

language = ECED8A16BE2A5E8871FD55F4842F16B1 

[CLS] Porque ele matou, esquartejou e deu os restos mortais da mãe do filho dele para os cachorros só pq não queria pagar pensã

De cara a l'anàlisi textual dels twits, només hem inclòs aquells en **anglès**, **castellà** i **català** filtrant mitjançant els codis corresponents als textos que hem identificat.

Per tant, només farem servir els languages `D3164C7FBCF2565DDF915B1B3AEFB1DC` (anglès), `06D61DCBBE938971E1EA0C38BD9B5446` (castellà) i `190BA7DA361BC06BC1D7E824C378064D` (català).

In [3]:
#Subset: English language 
language = 'D3164C7FBCF2565DDF915B1B3AEFB1DC'
d_eng = df[df.language == language].copy()
d_eng = d_eng.reset_index(drop=True)

#Subset: Spanish language 
language = '06D61DCBBE938971E1EA0C38BD9B5446'
d_cast = df[df.language == language].copy()
d_cast = d_cast.reset_index(drop=True)

#Subset: Catalan language 
language = '190BA7DA361BC06BC1D7E824C378064D'
d_cat = df[df.language == language].copy()
d_cat = d_cat.reset_index(drop=True)

print("English subset has a total of {} twits".format(d_eng.size))
print("Spanish subset has a total of {} twits".format(d_cast.size))
print("Catalan subset has a total of {} twits".format(d_cat.size))

English subset has a total of 4579224 twits
Spanish subset has a total of 999888 twits
Catalan subset has a total of 25272 twits


Una vegada filtrats els twits per idioma, parem atenció als tipus de twit:

In [4]:
d_eng['tweet_type'].value_counts()

TopLevel    99974
Retweet     72282
Quote       18545
Name: tweet_type, dtype: int64

Tenim tres tipus de twits:
- <b>TopLevel</b>: Twit publicat originalment per un usuari
- <b>Retweet</b>: Twit compartit per un usuari, però publicat originalment per un altre usuari
- <b>Quote</b>: Resposta d'un usuari a un TopLevel ó Retweet

Amb la intenció d'identificar contingut original per usuari, descartem els Retweets. 

In [5]:
engagement_user_type = 'engaged_with_user_id'
tweet_type = 'TopLevel'
d_eng_top_level = d_eng.loc[d_eng['tweet_type'] == tweet_type]

Així doncs, podem obtenir els usuaris amb més publicacions originals (TopLevel) de cada idioma.

In [6]:
d_eng_top_level[engagement_user_type].value_counts()[:10]

C6758D692A850E4C67B2763B66D1CFA8    659
5FF622786FB4924A067BD44D4B717570    509
E5D1B83B0E02FAFF871EEEF276D18132    395
7C03844E8B2E0C7B4346D41028AB14E2    328
FBB188A3C1E05C41587AAAC00B5B1879    232
F2A8BF0F4EB185E6D2E5E1A0DF4C33AE    204
416B919C0DAA48D42FF6780574034149    189
8A800256378089EF53C6F655F8690490    183
A9DAB08351D94BDE86235B37D6E8C61D    182
9D9C2BC354011249F2D4D9B9C4205AC9    168
Name: engaged_with_user_id, dtype: int64

Pel primer anàlisi, farem servir l'usuari més 'actiu' en anglès. 

In [7]:
#Take the user ID with most publications
engaging_user_id = d_eng_top_level[engagement_user_type].value_counts().index[0]

#Subset with only the twits of the user ID with most publications
d_eng_user = d_eng_top_level.loc[d_eng[engagement_user_type] == engaging_user_id]

Per tenir una idea del tipus d'usuari que hem obtingut, podem fer una ullada als seus 10 primers twits. 

In [8]:
for n in range (10): print(tokenizer.decode(d_eng_user['text_ tokens'].iloc[n]))

[CLS] Why didn [UNK] t John Bolton complain about this [UNK] nonsense [UNK] a long time ago, when he was very publicly terminated. He said, not that it matters, NOTHING! [SEP]
[CLS] [UNK] I like President Trump [UNK] s Tweets ( Social Media ), I like everything about him... and this Ukraine stuff, the trial, the impeachment, this isn [UNK] t t about Ukraine. Donald Trump has committed the two unpardonable sins in the eyes of the Democrats. He beat Hillary Clinton in 2016, and.. [SEP]
[CLS] Are you better off now than you were three years ago? Almost everyone say YES! [SEP]
[CLS] Reports are that basketball great Kobe Bryant and three others have been killed in a helicopter crash in California. That is terrible news! [SEP]
[CLS] The Democrat controlled House never even asked John Bolton to testify. It is up to them, not up to the Senate! [SEP]
[CLS]..... So, what the hell has happened to @ FoxNews. Only I know! Chris Wallace and others should be on Fake News CNN or MSDNC. How [UNK] s Sh

Podem veure que és un usuari de caire polític, molt segurament nord-americà. 

## Text tokens: anàlisi i data cleaning

Per poder entendre millor el model amb el que codifiquem i decodifiquem el text al nostre dataset, hem cregut oportú buscar si hi ha cap tipus d'ordre entre els tokens d'aquest. 

Imprimim els primers tokens del model per veure si es segueix algun ordre en la descripció de tokens. 

In [9]:
n = 1000
print(tokenizer.decode(x for x in range(0,n)))

[PAD] [unused1] [unused2] [unused3] [unused4] [unused5] [unused6] [unused7] [unused8] [unused9] [unused10] [unused11] [unused12] [unused13] [unused14] [unused15] [unused16] [unused17] [unused18] [unused19] [unused20] [unused21] [unused22] [unused23] [unused24] [unused25] [unused26] [unused27] [unused28] [unused29] [unused30] [unused31] [unused32] [unused33] [unused34] [unused35] [unused36] [unused37] [unused38] [unused39] [unused40] [unused41] [unused42] [unused43] [unused44] [unused45] [unused46] [unused47] [unused48] [unused49] [unused50] [unused51] [unused52] [unused53] [unused54] [unused55] [unused56] [unused57] [unused58] [unused59] [unused60] [unused61] [unused62] [unused63] [unused64] [unused65] [unused66] [unused67] [unused68] [unused69] [unused70] [unused71] [unused72] [unused73] [unused74] [unused75] [unused76] [unused77] [unused78] [unused79] [unused80] [unused81] [unused82] [unused83] [unused84] [unused85] [unused86] [unused87] [unused88] [unused89] [unused90] [unused91] [u

Veiem que els primers 105 tokens estan reservats per codis . Després s'assignen els tokens posteriors a  signes de puntuació i números. Desprès es codifiquen els distints alfabets que s'inclouen al model. 

Per un altre costat, analiztant tokens més endavant, podem veure que si a cada paraula reconeguda pel model, se li assignarà un token propi.

In [10]:
print(tokenizer.decode(x for x in range(11000,11200)))

33 годинеן three 1948 fuů invånaream kvadratkilometerou4 Earthä anche benот 1942는 made англria కి yeov 00 1957 người 1930 1920il used 1954mo dhe 09 vomwa Unjo sebagai formaésicabe Seion durante przez suchного deux 08 known ved 36 South ta nie then он USAken 1946 dass You Mariaina να 34 final 1955 से である Leagueной کی populationпраland का 1938lla than nuب 1953ine naamTسный 300 IV 1949 Thomas 1918 India τον 1952 All한 הוא State merupakanскойunні Oxford familie club July 1947 یکzione més elle Richardche June team Az 1936 unter học 44Pরneric care secondł zen ca mga Nel として Retrievedok відigth Marchو inomnuш She που には someada Santa there olehste GeoNamesut British pe Film ke West data5 gli areaán co 38 һәм 1937ció born 400 kao any became τωνdos vớitonского serie एक ook છેque vorку place Berlinныеty soorttes includingে


### Text cleaning

Si bé la resta de columnes del nostre dataset per ara seran tractades en altre moment i secció, el procés de neteja de les dades de text l'hem portat a terme a la par que l'anàlisi del text, tant tokenitzat com decodificat. 

Podem veure al text que es repeteixen uns codis a l'inici (`[CLS]`) i final (`[SEP]`) de cada twit. De cara al procès de data cleaning, ens interessaria eliminar aquests codis. Aprofitarem també per eliminar qualsevol possible aparició d'altres tokens que no aportaran valor al text: `[PAD]` i `[MASK]`. 

In [11]:
num_tokens = [0, 101, 102, 103]

for x in num_tokens:
    print('{}: {}'.format(x, tokenizer.decode(x)))

0: [ P A D ]
101: [ C L S ]
102: [ S E P ]
103: [ M A S K ]


Una vegada identificat el token per cadascun d'aquets codis, procedim a eliminar-los del nostre dataset.

In [12]:
#Convert the token list to string items
num_tokens = [ str(x) for x in num_tokens ]

#Remove all the items in num_tokens in each tweet
for k in range(len(d_eng['text_ tokens'])):
    for r in num_tokens:
        if r in d_eng['text_ tokens'][k]:
            d_eng['text_ tokens'][k].remove(r)

Podem veure la diferència imprimint els mateixos twits de l'usuari més actiu.

In [13]:
d_eng_top_level = d_eng.loc[d_eng['tweet_type'] == tweet_type]
engaging_user_id = d_eng_top_level[engagement_user_type].value_counts().index[0]
d_eng_user = d_eng_top_level.loc[d_eng[engagement_user_type] == engaging_user_id]

for n in range (10): print(tokenizer.decode(d_eng_user['text_ tokens'].iloc[n]))

Why didn [UNK] t John Bolton complain about this [UNK] nonsense [UNK] a long time ago, when he was very publicly terminated. He said, not that it matters, NOTHING!
[UNK] I like President Trump [UNK] s Tweets ( Social Media ), I like everything about him... and this Ukraine stuff, the trial, the impeachment, this isn [UNK] t t about Ukraine. Donald Trump has committed the two unpardonable sins in the eyes of the Democrats. He beat Hillary Clinton in 2016, and..
Are you better off now than you were three years ago? Almost everyone say YES!
Reports are that basketball great Kobe Bryant and three others have been killed in a helicopter crash in California. That is terrible news!
The Democrat controlled House never even asked John Bolton to testify. It is up to them, not up to the Senate!
..... So, what the hell has happened to @ FoxNews. Only I know! Chris Wallace and others should be on Fake News CNN or MSDNC. How [UNK] s Shep Smith doing? Watch, this will be the beginning of the end for 

## Links: enllaços, multimedia i retweets

### Accès als links

Per poder entendre amb més detall els twits, hem cregut necessari analitzar també els enllaços que s'hi publiquen en aquests. Malauradament, la decodificació dels tokens relatius als links resulta en les distintes parts de l'enllaç separades per un espai. 

Amb la idea de fer-los *clickables*, i així més fàcil d'accedir, reconstruirem els links a partir de les parts.

In [14]:
import re
#We are looking for patterns such as https : / / t. co / yvMa6bPqfy
valid_pattern = 'https : \/ \/ t\. co \/ [\dA-Za-z\.-]+'

Farem servir regex per identificar tots els patrons que corresponguin als links presents al text. Aprofitarem també per a guardar una columna amb el text decodificat.

Guardarem també els links en una columna apart, de manera que poguem accedir-hi independentment.

In [16]:
#Replace the links in the text in order to made them clickable

#Create a new column to store the list of links
d_eng['present_links_url'] = ""

#Now that the text will be more readable, we will create a column with the text
d_eng['text'] = ""    

for index, row in d_eng.iterrows():
    links_list = []
    links_list_clean = []
    tweet_text = tokenizer.decode(d_eng['text_ tokens'][index])
    links_list = re.findall(valid_pattern, tweet_text)
    
    for i, s in enumerate(links_list):
        if i != 0:
            tweet_text = tweet_text_replaced
        link_old = s
        link_new = s.replace(' ','') # I use replace because s.strip() is not working
        links_list_clean.append(link_new)
        tweet_text_replaced = tweet_text.replace(link_old, link_new)
    
    d_eng.at[index, 'present_links_url'] = links_list_clean
    
    if links_list == []:    
        d_eng.at[index, 'text'] = tweet_text
    else:
        d_eng.at[index, 'text'] = tweet_text_replaced

Per tal de agrupar totes les característiques textuals, reorganitzem les columnes del dataset.

In [17]:
second_col = d_eng['text']
d_eng.drop(labels=['text'], axis=1, inplace = True)
d_eng.insert(1, 'text', second_col)
d_eng.head()

col = d_eng['present_links_url']
d_eng.drop(labels=['present_links_url'], axis=1, inplace = True)
d_eng.insert(4, 'present_links_url', col)
d_eng.head()

,text_ tokens,text,hashtags,tweet_id,present_links_url,present_media,present_links,present_domains,tweet_type,language,...,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engaging_user_account_creation,engagee_follows_engager,reply_timestamp,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp
0,"[16493, 12478, 117, 10105, 42370, 76299, 100, ...","Jacob Young, the Syndicate [UNK] s Director of...",NaN,39024FBE0136E046D1357196BAECFCA6,[https://t.co/g5evrQa8va],GIF,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,...,000006829BEADA9EEA695CF0C334B426,2,29,False,1568107028,False,NaN,NaN,NaN,NaN
1,"[14924, 16118, 10114, 12888, 15127, 31204, 101...",Final week to see my exhibition in the Shircli...,3653868A966576CF17D6A9064889BCED\t7A6710E791A1...,AB3ADBBD011F88D10FE7F6C5FDAB214C,[https://t.co/PxG5XzTruM],Photo,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,...,000269A50293CC2B1E9C72B287FA624E,515,424,False,1254588643,True,NaN,NaN,NaN,1.581090e+09
2,"[138, 19826, 10108, 10105, 10635, 10105, 12250...",A lot of the time the side that wins is simply...,NaN,675D7920EA2FB4869BA767F5122FB115,[],NaN,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,...,0004C10C480EF88F3EED5822149DC93F,4,125,False,1547955941,False,NaN,NaN,NaN,1.581135e+09
3,"[56898, 137, 42374, 36630, 10797, 90861, 131, ...","RT @ readyletsgo27 : Schiff, Nadler and Pelosi...",NaN,7EAAE5C243D675AD4CFC2C052B0E0BB5,[],NaN,NaN,NaN,Retweet,D3164C7FBCF2565DDF915B1B3AEFB1DC,...,000830B836532B8FC5564949E94FEDD6,33439,35598,False,1261862258,True,NaN,1.581012e+09,NaN,NaN
4,"[137, 87043, 10679, 11369, 11211, 22650, 11305...",@ LiliyaL62076921 Unique beauty of flowers [UN...,C8F2526CD1AEA3A8CFC67EB4B79E46D5,E243DF6401359C38C5D21B6FB86A881E,[],NaN,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,...,0011F45B80474E4FF47250ED24C5482C,27980,25991,False,1493066269,True,NaN,NaN,NaN,1.581058e+09


Finalment, imprimim 10 twits a l'atzar del mateix usuari que del que hem estat llegint fins ara. Hem inclòs el twit original (ja netejat de caràcters no desitjats) per comprovar que tot el procès ha sigut correcte fins ara. 

In [18]:
#We'll use a list of 10 random ints in order to print 10 random tweets 
import random
randomlist = []
for i in range(0,19):
    n = random.randint(0,100)
    randomlist.append(n)

In [19]:
d_eng_top_level = d_eng.loc[d_eng['tweet_type'] == tweet_type]
engaging_user_id = d_eng_top_level[engagement_user_type].value_counts().index[0]
d_eng_user = d_eng_top_level.loc[d_eng[engagement_user_type] == engaging_user_id]

for n in randomlist: 
    links_list = d_eng_user['present_links_url'].iloc[n]
#    print(d_eng_user['engaged_with_user_id'].iloc[n] + ': \n\n'  + 
#                           'Original text: \n' + tokenizer.decode(d_eng_user['text_ tokens'].iloc[n]) + '\n\n' +
#                           'Resulting text: \n' + d_eng_user['text'].iloc[n] + '\n\n' +
#                           'Links: ' + '{} \n'.format(d_eng_user['present_links_url'].iloc[n]) + '\n\n' +
#                           '---------------' + '\n\n' )
    print('User ID: ' + d_eng_user['engaged_with_user_id'].iloc[n] + '\n'  +
          'Tweet ID: ' + d_eng_user['tweet_id'].iloc[n] + '\n\n'  +
          'Original text: \n' + tokenizer.decode(d_eng_user['text_ tokens'].iloc[n]) + '\n\n' +
          'Resulting text: \n' + d_eng_user['text'].iloc[n] + '\n')
    if links_list != []:
        print ('Links: ')
        for item in links_list:
            print(item)
    print('------------------------ \n')

User ID: C6758D692A850E4C67B2763B66D1CFA8
Tweet ID: 842C962ACCB4EA846EEDE00514B5234C

Original text: 
With Votes in the House tomorrow, Democrats want to make it harder for Presidents to defend America, and stand up to, as an example, Iran. Protect our GREAT COUNTRY!

Resulting text: 
With Votes in the House tomorrow, Democrats want to make it harder for Presidents to defend America, and stand up to, as an example, Iran. Protect our GREAT COUNTRY!

------------------------ 

User ID: C6758D692A850E4C67B2763B66D1CFA8
Tweet ID: 86FDD77AA2C0C610056C089631CDF7B7

Original text: 
Washington Dems have spent the last 3 years trying to overturn the last election [UNK] and we will make sure they face another crushing defeat in the NEXT ELECTION. Together, we are going to win back the House, we are going to hold the Senate, & amp ; we are going to keep the White House! https : / / t. co / VshQceiwUA

Resulting text: 
Washington Dems have spent the last 3 years trying to overturn the last electio

Veiem que ara tots els links presents al text són clickables i hi podem accedir directament des d'aquest notebook. 

### Links i multimedia

Hem observat que tots els twits que inclouen multimedia (imagtes, GIFs o vídeos), inclouen un link al propit twit. Això ens permet accedir fàcilment als arxius que comparteixen els usuaris i contextualitzar encara més el contingut del seu twit. 

In [20]:
d_eng['present_media'].fillna("No Media", inplace=True)

#Checking tweets with more than one link
for i in range(0,100):
    if len(d_eng['present_links_url'].iloc[i]) > 1:
        if d_eng['present_media'].iloc[i]:
            print(d_eng['present_media'].iloc[i] + '\n')
        print(d_eng['text'].iloc[i] + '\n')
        

Video

[UNK] This is a very fast moving, constantly changing situation, [UNK] HHS Sec. Azar says about the # coronavirusoutbreak. He says the HHS is working to understand the speed of spread and the severity of the virus. https://t.co/qZuyJkklqg https://t.co/4hiiozzgFH

Video

A Morning in Paris... ♥. https://t.co/jaJduGn5FQ https://t.co/g3lsTIL7Xv

Photo	Photo	Photo	Photo

NEW! We've teamed up with @ STSPhonoCo to bring an exclusive color variant of Castlevania : The Adventure ReBirth to the Yeteemart! It's got killer tracks from Castlevania, Castlevania III, Dracula X, and more, and features art by @ Dreweyes! https://t.co/hoflXYORIv https://t.co/yuBRQhh4D9

Video

From breathtaking beaches to friendly locals, there's so much to love about the DR! @ DarlingEscapes is sharing about her beach vacay, plus the best cities to visit in the Dominican Republic : https://t.co/HU7XgZO9Of https://t.co/XeynSRY70w

Photo

RT @ soompi : LISTEN : # LOONA Previews Mini Album " # " With Highlight Med

In [24]:
#Checking links with No Media
for i in range(0,3000):
    if len(d_eng['present_links_url'].iloc[i]) > 1:
        if d_eng['present_media'].iloc[i] == 'No Media':
            print(d_eng['present_media'].iloc[i] + '\n')
            print(d_eng['text'].iloc[i] + '\n')

No Media

RT @ reflectivelatex : Tattoos, transparent latex and vacbeds. Oh my! With @ thecamdamage. Full video :.. https://t.co/xABJG6K1OD https://t.co/j [UNK]

No Media

[UNK] By Popular [UNK] This Hoodie Sweater Dress is so Bomb!. [UNK] Now ~ Pay [UNK]. https://t.co/z4ezXTSPRB......... [UNK] Online and Schedule Your Houston Showroom Pick Up........... # dresses # hoodiedresses [UNK] https://t.co/dPPsizsA8V

No Media

Tonight is our @ iHeartRadio LIVE show presented by HOT TOPIC! Watch on @ thecw app or on https://t.co/PsYe4Fqk9f at 9pm ET / 6pm PT! # iHeartBTS. https://t.co/gWsJqt4GxC

No Media

RT @ itskarinareyes : My shop has been updated! Go check it out, I have a lot of cute things on https://t.co/elYAuf4uT8 [UNK] https://t.co/BLbRRb [UNK]

No Media

RT @ wizkhalifa : If you party with me you party with @ McQueenVF @ TylaYaweh. # HighRightNowRemix. [UNK] : https://t.co/jKD4Qz3MxX https://t.co/NYuOb [UNK]

No Media

The Scenic Route to York feat : Night Heron Soundtrack https://

In [22]:
d_eng['link_to_media'] = ""

for index, row in d_eng.iterrows():
    if row['present_links_url'] != []:
        d_eng.at[index, 'link_to_media'] = row['present_links_url'][-1]

col = d_eng['link_to_media']
d_eng.drop(labels=['link_to_media'], axis=1, inplace = True)
d_eng.insert(7, 'link_to_media', col)            
            
d_eng.head()

,text_ tokens,text,hashtags,tweet_id,present_links_url,present_media,present_links,link_to_media,present_domains,tweet_type,...,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engaging_user_account_creation,engagee_follows_engager,reply_timestamp,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp
0,"[16493, 12478, 117, 10105, 42370, 76299, 100, ...","Jacob Young, the Syndicate [UNK] s Director of...",NaN,39024FBE0136E046D1357196BAECFCA6,[https://t.co/g5evrQa8va],GIF,NaN,https://t.co/g5evrQa8va,NaN,TopLevel,...,000006829BEADA9EEA695CF0C334B426,2,29,False,1568107028,False,NaN,NaN,NaN,NaN
1,"[14924, 16118, 10114, 12888, 15127, 31204, 101...",Final week to see my exhibition in the Shircli...,3653868A966576CF17D6A9064889BCED\t7A6710E791A1...,AB3ADBBD011F88D10FE7F6C5FDAB214C,[https://t.co/PxG5XzTruM],Photo,NaN,https://t.co/PxG5XzTruM,NaN,TopLevel,...,000269A50293CC2B1E9C72B287FA624E,515,424,False,1254588643,True,NaN,NaN,NaN,1.581090e+09
2,"[138, 19826, 10108, 10105, 10635, 10105, 12250...",A lot of the time the side that wins is simply...,NaN,675D7920EA2FB4869BA767F5122FB115,[],No Media,NaN,,NaN,TopLevel,...,0004C10C480EF88F3EED5822149DC93F,4,125,False,1547955941,False,NaN,NaN,NaN,1.581135e+09
3,"[56898, 137, 42374, 36630, 10797, 90861, 131, ...","RT @ readyletsgo27 : Schiff, Nadler and Pelosi...",NaN,7EAAE5C243D675AD4CFC2C052B0E0BB5,[],No Media,NaN,,NaN,Retweet,...,000830B836532B8FC5564949E94FEDD6,33439,35598,False,1261862258,True,NaN,1.581012e+09,NaN,NaN
4,"[137, 87043, 10679, 11369, 11211, 22650, 11305...",@ LiliyaL62076921 Unique beauty of flowers [UN...,C8F2526CD1AEA3A8CFC67EB4B79E46D5,E243DF6401359C38C5D21B6FB86A881E,[],No Media,NaN,,NaN,TopLevel,...,0011F45B80474E4FF47250ED24C5482C,27980,25991,False,1493066269,True,NaN,NaN,NaN,1.581058e+09


### Eliminació de links als tokens

Com els links i les urls no es codifiquen amb un sol token, hem preferit prescindir dels tokens referents als enllaços i crear una nova columna de tokens sense aquesta informació.

In [25]:
for index, row in d_eng.iterrows():
    links_list = []
    tweet_text = d_eng['text'][index]
    links_list = re.findall(valid_pattern, tweet_text)
    
    for i, s in enumerate(links_list):
        if i != 0:
            tweet_text = tweet_text_replaced
        link_old = s
        tweet_text_replaced = tweet_text.replace(link_old, '')
    
    if links_list == []:
        d_eng.at[index, 'text_ tokens'] = d_eng['text_ tokens'][index]        
    else:
        new_tokens = tokenizer.encode(tweet_text_replaced)
        new_tokens = [str(x) for x in new_tokens]
        d_eng.at[index, 'text_ tokens'] = new_tokens

for k in range(len(d_eng['text_ tokens'])):
    for r in num_tokens:
        if r in d_eng['text_ tokens'][k]:
            d_eng['text_ tokens'][k].remove(r)     
            
d_eng.head()

,text_ tokens,text,hashtags,tweet_id,present_links_url,present_media,present_links,link_to_media,present_domains,tweet_type,...,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engaging_user_account_creation,engagee_follows_engager,reply_timestamp,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp
0,"[16493, 12478, 117, 10105, 42370, 76299, 100, ...","Jacob Young, the Syndicate [UNK] s Director of...",NaN,39024FBE0136E046D1357196BAECFCA6,[https://t.co/g5evrQa8va],GIF,NaN,https://t.co/g5evrQa8va,NaN,TopLevel,...,000006829BEADA9EEA695CF0C334B426,2,29,False,1568107028,False,NaN,NaN,NaN,NaN
1,"[14924, 16118, 10114, 12888, 15127, 31204, 101...",Final week to see my exhibition in the Shircli...,3653868A966576CF17D6A9064889BCED\t7A6710E791A1...,AB3ADBBD011F88D10FE7F6C5FDAB214C,[https://t.co/PxG5XzTruM],Photo,NaN,https://t.co/PxG5XzTruM,NaN,TopLevel,...,000269A50293CC2B1E9C72B287FA624E,515,424,False,1254588643,True,NaN,NaN,NaN,1.581090e+09
2,"[138, 19826, 10108, 10105, 10635, 10105, 12250...",A lot of the time the side that wins is simply...,NaN,675D7920EA2FB4869BA767F5122FB115,[],No Media,NaN,,NaN,TopLevel,...,0004C10C480EF88F3EED5822149DC93F,4,125,False,1547955941,False,NaN,NaN,NaN,1.581135e+09
3,"[56898, 137, 42374, 36630, 10797, 90861, 131, ...","RT @ readyletsgo27 : Schiff, Nadler and Pelosi...",NaN,7EAAE5C243D675AD4CFC2C052B0E0BB5,[],No Media,NaN,,NaN,Retweet,...,000830B836532B8FC5564949E94FEDD6,33439,35598,False,1261862258,True,NaN,1.581012e+09,NaN,NaN
4,"[137, 87043, 10679, 11369, 11211, 22650, 11305...",@ LiliyaL62076921 Unique beauty of flowers [UN...,C8F2526CD1AEA3A8CFC67EB4B79E46D5,E243DF6401359C38C5D21B6FB86A881E,[],No Media,NaN,,NaN,TopLevel,...,0011F45B80474E4FF47250ED24C5482C,27980,25991,False,1493066269,True,NaN,NaN,NaN,1.581058e+09


Així, si en un futur volem fer servir els tokens amb un embedding per realitzar una classificació, no afegirem dades redundants. 

### Hashtags i mencions

De manera similar al que hem fet amb els links, també hem extret les mencions i els hashtags presents a cadascun dels twtits. En aquest cas no els hem el·liminat del text.

#### Hashtags

In [26]:
#Replace the hashtags in the text 
valid_pattern = '# [a-zA-Z0-9]+'

#Create a new column to store the list of links
d_eng['hashtags_text'] = ""

for index, row in d_eng.iterrows():
    hashtags_list = []
    hashtags_list = re.findall(valid_pattern, d_eng['text'][index])
    for i, s in enumerate(hashtags_list):
        #d_eng['text'][index] = d_eng['text'][index].replace(s,'')
        hashtags_list[i] = s.replace(' ','') # I use replace because s.strip() is not working 
        
    d_eng.at[index, 'hashtags_text'] = hashtags_list
    #d_eng['text'][index] = re.sub(valid_pattern, '', d_eng['text'][index])    
    
col = d_eng['hashtags_text']
d_eng.drop(labels=['hashtags_text'], axis=1, inplace = True)
d_eng.insert(3, 'hashtags_text', col)            
            
d_eng.head()

,text_ tokens,text,hashtags,hashtags_text,tweet_id,present_links_url,present_media,present_links,link_to_media,present_domains,...,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engaging_user_account_creation,engagee_follows_engager,reply_timestamp,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp
0,"[16493, 12478, 117, 10105, 42370, 76299, 100, ...","Jacob Young, the Syndicate [UNK] s Director of...",NaN,[],39024FBE0136E046D1357196BAECFCA6,[https://t.co/g5evrQa8va],GIF,NaN,https://t.co/g5evrQa8va,NaN,...,000006829BEADA9EEA695CF0C334B426,2,29,False,1568107028,False,NaN,NaN,NaN,NaN
1,"[14924, 16118, 10114, 12888, 15127, 31204, 101...",Final week to see my exhibition in the Shircli...,3653868A966576CF17D6A9064889BCED\t7A6710E791A1...,"[#vincennesuniversity, #shircliffgallery, #ins...",AB3ADBBD011F88D10FE7F6C5FDAB214C,[https://t.co/PxG5XzTruM],Photo,NaN,https://t.co/PxG5XzTruM,NaN,...,000269A50293CC2B1E9C72B287FA624E,515,424,False,1254588643,True,NaN,NaN,NaN,1.581090e+09
2,"[138, 19826, 10108, 10105, 10635, 10105, 12250...",A lot of the time the side that wins is simply...,NaN,[],675D7920EA2FB4869BA767F5122FB115,[],No Media,NaN,,NaN,...,0004C10C480EF88F3EED5822149DC93F,4,125,False,1547955941,False,NaN,NaN,NaN,1.581135e+09
3,"[56898, 137, 42374, 36630, 10797, 90861, 131, ...","RT @ readyletsgo27 : Schiff, Nadler and Pelosi...",NaN,[],7EAAE5C243D675AD4CFC2C052B0E0BB5,[],No Media,NaN,,NaN,...,000830B836532B8FC5564949E94FEDD6,33439,35598,False,1261862258,True,NaN,1.581012e+09,NaN,NaN
4,"[137, 87043, 10679, 11369, 11211, 22650, 11305...",@ LiliyaL62076921 Unique beauty of flowers [UN...,C8F2526CD1AEA3A8CFC67EB4B79E46D5,[#BeHappy],E243DF6401359C38C5D21B6FB86A881E,[],No Media,NaN,,NaN,...,0011F45B80474E4FF47250ED24C5482C,27980,25991,False,1493066269,True,NaN,NaN,NaN,1.581058e+09


#### Mencions

In [27]:
#Replace the hashtags in the text 
valid_pattern = '@ [a-zA-Z0-9]+'

#Create a new column to store the list of links
d_eng['mentions_text'] = ""

for index, row in d_eng.iterrows():
    mentions_list = []
    mentions_list = re.findall(valid_pattern, tokenizer.decode(d_eng['text_ tokens'][index]))
    for i, s in enumerate(mentions_list):
        mentions_list[i] = s.replace(' ','') # I use replace because s.strip() is not working 
        
    d_eng.at[index, 'mentions_text'] = mentions_list
    #d_eng['text'].remove(s for s in enumerate(mentions_list))

col = d_eng['mentions_text']
d_eng.drop(labels=['mentions_text'], axis=1, inplace = True)
d_eng.insert(4, 'mentions_text', col)            
            
d_eng.head()

,text_ tokens,text,hashtags,hashtags_text,mentions_text,tweet_id,present_links_url,present_media,present_links,link_to_media,...,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engaging_user_account_creation,engagee_follows_engager,reply_timestamp,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp
0,"[16493, 12478, 117, 10105, 42370, 76299, 100, ...","Jacob Young, the Syndicate [UNK] s Director of...",NaN,[],[],39024FBE0136E046D1357196BAECFCA6,[https://t.co/g5evrQa8va],GIF,NaN,https://t.co/g5evrQa8va,...,000006829BEADA9EEA695CF0C334B426,2,29,False,1568107028,False,NaN,NaN,NaN,NaN
1,"[14924, 16118, 10114, 12888, 15127, 31204, 101...",Final week to see my exhibition in the Shircli...,3653868A966576CF17D6A9064889BCED\t7A6710E791A1...,"[#vincennesuniversity, #shircliffgallery, #ins...",[],AB3ADBBD011F88D10FE7F6C5FDAB214C,[https://t.co/PxG5XzTruM],Photo,NaN,https://t.co/PxG5XzTruM,...,000269A50293CC2B1E9C72B287FA624E,515,424,False,1254588643,True,NaN,NaN,NaN,1.581090e+09
2,"[138, 19826, 10108, 10105, 10635, 10105, 12250...",A lot of the time the side that wins is simply...,NaN,[],[],675D7920EA2FB4869BA767F5122FB115,[],No Media,NaN,,...,0004C10C480EF88F3EED5822149DC93F,4,125,False,1547955941,False,NaN,NaN,NaN,1.581135e+09
3,"[56898, 137, 42374, 36630, 10797, 90861, 131, ...","RT @ readyletsgo27 : Schiff, Nadler and Pelosi...",NaN,[],[@readyletsgo27],7EAAE5C243D675AD4CFC2C052B0E0BB5,[],No Media,NaN,,...,000830B836532B8FC5564949E94FEDD6,33439,35598,False,1261862258,True,NaN,1.581012e+09,NaN,NaN
4,"[137, 87043, 10679, 11369, 11211, 22650, 11305...",@ LiliyaL62076921 Unique beauty of flowers [UN...,C8F2526CD1AEA3A8CFC67EB4B79E46D5,[#BeHappy],[@LiliyaL62076921],E243DF6401359C38C5D21B6FB86A881E,[],No Media,NaN,,...,0011F45B80474E4FF47250ED24C5482C,27980,25991,False,1493066269,True,NaN,NaN,NaN,1.581058e+09


## Anàlisi dels diferents tipus de twit 

Com a últim pas del nostre anàlisi, podem fer una caracterització qualitativa dels twits de diferents tipus: **TopLevel**, **Retweet** i **Quote**.

Per això mostrem 20 twits de cada tipus pel nostre usuari target. 

In [28]:
tweet_type = 'TopLevel'
d_eng_toplevel = d_eng.loc[d_eng['tweet_type'] == tweet_type]
engaging_user_id = d_eng_toplevel[engagement_user_type].value_counts().index[0]
d_eng_user = d_eng_toplevel.loc[d_eng[engagement_user_type] == engaging_user_id]

for n in range (20): print(d_eng_user['text'].iloc[n])

Why didn [UNK] t John Bolton complain about this [UNK] nonsense [UNK] a long time ago, when he was very publicly terminated. He said, not that it matters, NOTHING!
[UNK] I like President Trump [UNK] s Tweets ( Social Media ), I like everything about him... and this Ukraine stuff, the trial, the impeachment, this isn [UNK] t t about Ukraine. Donald Trump has committed the two unpardonable sins in the eyes of the Democrats. He beat Hillary Clinton in 2016, and..
Are you better off now than you were three years ago? Almost everyone say YES!
Reports are that basketball great Kobe Bryant and three others have been killed in a helicopter crash in California. That is terrible news!
The Democrat controlled House never even asked John Bolton to testify. It is up to them, not up to the Senate!
..... So, what the hell has happened to @ FoxNews. Only I know! Chris Wallace and others should be on Fake News CNN or MSDNC. How [UNK] s Shep Smith doing? Watch, this will be the beginning of the end for 

Els twits *TopLevel*, al ser twits originals, acostumen a seguir l'estil propi de l'usuari que els publica. El text sol ser extens (dins el que ho permet una plataforma de microblogging) i contextualitzat (l'usuari introdueix mínimament el tema del que parla). Són els twits d'on més informació a primera vista podem extreure del text.

In [29]:
tweet_type = 'Retweet'
d_eng_retweet = d_eng.loc[d_eng['tweet_type'] == tweet_type]
engaging_user_id = d_eng_retweet[engagement_user_type].value_counts().index[0]
d_eng_user = d_eng_retweet.loc[d_eng[engagement_user_type] == engaging_user_id]

for n in range (20): print(d_eng_user['text'].iloc[n])

RT @ brithume : McConnell had a choice, to be liked by Democrats and the media or to be effective. He chose effective. By @ wjmcgurn https : [UNK]
RT @ JennaEllisEsq : After everything @ realDonaldTrump has done for this country ( at great personal cost! ) it is so sad that so many are will [UNK]
RT @ SenThomTillis : The only thing we [UNK] ve seen from the # impeachment trial is a political exercise by Chuck and Nancy to prevent us from cont [UNK]
RT @ WhiteHouse : " I am concerned about lowering impeachment standards to fit a paucity of evidence and an abundance of anger. " [UNK] Jonathan Tu [UNK]
RT @ Jim _ Jordan : 2 hours of truth beats over 20 hours of presumptions, assumptions, and hearsay.
RT @ realDonaldTrump : All Democrats should watch this! https://t.co/WFK33pR0Lv
RT @ SteveScalise : President Trump's legal team completely destroyed 3 days of Adam Schiff's false allegations in just 2 hours. Today was de [UNK]
RT @ SenateGOP : Chairman @ SenJohnBarrasso :.. The American peop

Els *Retweets* són els textos més diversos, ja que gairebé sempre són textos *TopLevel* creats per altres usuaris. No obstant, lo normal és que vagin alineats amb els temes que es tracten als twits *TopLevel* de l'usuari. 

In [30]:
tweet_type = 'Quote'
d_eng_quote = d_eng.loc[d_eng['tweet_type'] == tweet_type]
engaging_user_id = d_eng_quote[engagement_user_type].value_counts().index[0]
d_eng_user = d_eng_quote.loc[d_eng[engagement_user_type] == engaging_user_id]

for n in range (20): print(d_eng_user['text'].iloc[n])

Why didn [UNK] t Schiff release this?
True!
Don Lemon, the dumbest man on television ( with terrible ratings! ).
Incredible numbers!
Incredible numbers!
Nadler ripped final argument away from Schiff, thinks Shifty did a terrible job. They are fighting big time!
Thank you!
Charlie, I always knew you are brilliant!
Thank you Roger, it is a big Hoax!
Such a great family!
Don Lemon, the dumbest man on television ( with terrible ratings! ).
Democrats admitting the Impeachment Hoax!
Thank you to Chuck & amp ; all. USMCA is going to be GREAT!
True!
They will never be satisfied!
A very good question!
Congrats to Christine!
Thank you to Chuck & amp ; all. USMCA is going to be GREAT!
They will never be satisfied!
Incredible numbers!


En aquest cas podem veure que el text s'apropa més al d'un twit *TopLevel*, amb contingut original de l'usuari. En canvi, les mencions acostumen a ser textos més curts que els twits originals i amb menys contexte, al ser una rèplica i estar referint-se a un altre text. 

Arribat a aquest punt, donem per complet el nostre anàlisi d'alt nivell i acabar d'extreure conclusions que condicionen els propers pasos a l'hora de dissenyar un sistema de recomanació.

Obviament, aquesta caracterització del text podria ser molt més exhaustiva. Entre altres aspectes, només hem fet servir l'usuari més actiu del nostre dataset (que, en base al links multimedia i el tipus de twit, hem podet identificar que és el compte oficial de Donald Trump, presidentes dels EUA). 

Pel nostre anàlisi del dataset també ens hem basat en el nostre coneixement previ de l'estructura, tipus de publicacions i idiosincràsia de Twitter. Hem obviat l'anàlisi d'altres aspectes (com altres usuaris o llengües) per tal de no fer aquesta secció -encara més- extensa. 